# GRASS GIS 101: Hop into the Fast Lane with Notebooks!

Welcome to Jupyter Notebooks! In these notebooks, we'll repeat some of the same workflows we did in the GUI but using code. Then, we'll take it a step further and create our own custom GRASS script!

In this first part, we will:
1. [learn the basics of operating in a notebook environment](#1.-The-Notebook-environment)
2. [Start GRASS GIS and create a new project](#2.-Import-Libraries)
3. [Import and visualize data](#4.-Importing-data)

***

## 1. The Notebook environment

By default all cells are running Python:

In [ ]:
import sys
v = sys.version_info
print(f"We are using Python {v.major}.{v.minor}.{v.micro}")

We can also use `!` to run individual lines in the terminal.

In [ ]:
!echo Howdy

Here are some useful keyboard shortcuts in notebooks:

* `shift - enter` execute cell
* `alt - enter` execute cell and insert new below
* `esc` exit cursor/edit mode and enter command mode
* `a` add cell above
* `b` add cell below
* `dd` delete cell
* `x` cut selected cells
* `c` copy selected cells
* `v` paste cells below
* `m` change cell to Markdown
* `y` change cell to Code

Try a few below!

Markdown cells (such as this one) don't execute code but they **can** contain _nice_ formatting.

They can also include `code` snippets:

```
def hello(name):
    print(f"hello {name}")
```

# And Titles
## Headings
### Subheadings
#### and Sub-subheadings

<div class="alert alert-info">
... and HTML formatting
</div>


... and even LaTex!

$
f(x) = \int_{-\infty}^{\infty} e^{-x^2} dx
$

***

## 2. Import Libraries

Import Python standard library and IPython packages we need.

In [ ]:
import subprocess
import sys
from pathlib import Path

We're going to import the GRASS GIS python API (`grass.script`) and the GRASS GIS Jupyter package (`grass.jupyter`). But first, we need to find the path to those packages using the `--config python_path` command. We use `subprocess.check_output` to find the path and `sys.path.append` to add it to the path.

In [ ]:
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True, shell=False).strip()
)

And now we can import the GRASS python packages!

In [ ]:
# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

**We'll have to repeat the commands above every time we want to start a GRASS session in a notebook.**

***
## 3. Create a new project and start GRASS session

In [ ]:
gs.create_project("NC_Sentinel", epsg=3358, overwrite=True) # Warning! Overwrite=True will delete any all existing data in the project! Use carefully...

In [ ]:
# Start GRASS Session
session = gj.init("./NC_Sentinel/PERMANENT")

We've launched GRASS GIS now! We can access GRASS GIS commands using the command line interface (with the `!` line magic):

In [ ]:
!g.version

In [ ]:
!g.region -p

In [ ]:
!r.univar --help

***
## 4. Importing data

<div class="alert alert-info">
Download <a href="https://grass.osgeo.org/sampledata/north_carolina/nc_sentinel_utm17n.zip">these Sentinel-2 images</a>. Put the unzipped download in this directory.
</div>

Since we have several files to import, we can make a Python for-loop to import them all.

In [ ]:
files = sorted(Path('./nc_sentinel_utm17n/S2A_MSIL2A_20220304T160151_N0400_R097_T17SQV_20220304T215812.SAFE/GRANULE/L2A_T17SQV_A034986_20220304T160221/IMG_DATA/R10m').glob('*.jp2'))
files

In [ ]:
for file in files[1:]:
    name = str(file)[-11:-4]
    print("importing " + name)
    gs.run_command("r.import", input=file, output=name, resolution="value", resolution_value=10)

Each of these rasters corresponds to a Sentinel-2 band.

| Raster | Band |
|------------|-----------|
| B02_10m | Blue |
| B03_10m | Green |
| B04_10m | Red |
| B08_10m | NIR |


Let's look at the available data in our location:

In [ ]:
!g.list type=raster,vector -m -t

In [ ]:
!r.info map=B02_10m

In [ ]:
!g.region raster=B02_10m -p

***
## 5. Data Visualization with `grass.jupyter`

`grass.jupyter.Map()` creates and displays GRASS maps as PNG images. `gj.Map()` accepts any GRASS display module as a method by replacing the "." with "\_" in the module name. For example:

In [ ]:
example = gj.Map()
example.d_rast(map="B02_10m") # d.rast map=B02_10m
example.show()

In [ ]:
interactive_example = gj.InteractiveMap(width=800)
interactive_example.add_raster("B02_10m")
interactive_example.show()

In [ ]:
interactive_example.add_layer_control()
interactive_example.show()